In [1]:
import logging
import os
from fetch_data import *
from forward_integration import *
from initial_states import *
import pandas as pd
import numpy as np

curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)
erva_pop_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')
print(curr_dir)

/Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta


In [2]:
infectios_age_finland, infectios_age_finland_prop = fetch_finland_cases_age_weekly(logger, 9)
infectios_age_finland.loc[infectios_age_finland['Time'] == 'Year 2021 Week 15']

,Time,age_group,val
670,Year 2021 Week 15,0-9,240.0
671,Year 2021 Week 15,10-19,310.0
672,Year 2021 Week 15,20-29,354.0
673,Year 2021 Week 15,30-39,354.0
674,Year 2021 Week 15,40-49,295.0
675,Year 2021 Week 15,50-59,200.0
676,Year 2021 Week 15,60-69,102.0
677,Year 2021 Week 15,70-79,44.0
678,Year 2021 Week 15,80+,31.0


In [3]:
infectios_age_finland_prop.loc[infectios_age_finland_prop['Time'] == 'Year 2021 Week 15']

,Time,age_group,val
670,Year 2021 Week 15,0-9,0.124352
671,Year 2021 Week 15,10-19,0.160622
672,Year 2021 Week 15,20-29,0.183420
673,Year 2021 Week 15,30-39,0.183420
674,Year 2021 Week 15,40-49,0.152850
675,Year 2021 Week 15,50-59,0.103627
676,Year 2021 Week 15,60-69,0.052850
677,Year 2021 Week 15,70-79,0.022798
678,Year 2021 Week 15,80+,0.016062


In [4]:
rounded_vals = infectios_age_finland_prop.loc[infectios_age_finland_prop['Time'] == 'Year 2021 Week 15'].val
np.round(rounded_vals, 4)

670    0.1244
671    0.1606
672    0.1834
673    0.1834
674    0.1528
675    0.1036
676    0.0528
677    0.0228
678    0.0161
Name: val, dtype: float64

In [5]:
hosp_by_erva = fetch_hs_hospitalizations(logger)
hosp_by_erva.loc[hosp_by_erva['date'] == '2021-04-19', ]

,date,erva,hospitalized,ward,icu,dead
1481,2021-04-19,HYKS,109,88,21,578
1482,2021-04-19,KYS,10,5,5,81
1483,2021-04-19,OYS,11,11,0,55
1484,2021-04-19,TAYS,19,17,2,79
1485,2021-04-19,TYKS,17,11,6,98


In [6]:
cases_erva = fetch_thl_cases_erva_daily(logger)

In [7]:
cases_erva.loc[(cases_erva['Time'] >= '2021-04-12') & (cases_erva['Time'] <= '2021-04-18')].groupby('erva').sum()

,val
erva,
HYKS,1179.0
KYS,80.0
OYS,76.0
TAYS,225.0
TYKS,347.0
Åland,23.0


In [8]:
t0 = '2021-04-18'
number_age_groups = 9
num_ervas = 5
erva_pop_file = 'stats/erva_population_age_2020.csv'

pop_ervas_age, _ = static_population_erva_age(logger, erva_pop_file,
                                              number_age_groups=number_age_groups)
pop_ervas_age = pop_ervas_age[~pop_ervas_age['erva'].str.contains('All')]
pop_ervas_age = pop_ervas_age[~pop_ervas_age['erva'].str.contains('land')]
pop_ervas_age = pop_ervas_age.sort_values(['erva', 'age_group'])
pop_ervas_npy = pop_ervas_age['Total'].values
pop_ervas_npy = pop_ervas_npy.reshape(num_ervas, number_age_groups)

ervas_order = EPIDEMIC['ervas_order']

ervas_df = list(pd.unique(pop_ervas_age['erva']))
ervas_pd_order = [ervas_df.index(erva) for erva in ervas_order]
# Rearrange rows in the correct order
age_er = pop_ervas_npy[ervas_pd_order, :]

N_p = num_ervas
N_g = number_age_groups
csv_name = 'out/epidemic_finland_%d.csv' % (number_age_groups, )

# Reading CSV
epidemic_csv = pd.read_csv(csv_name)
# Getting only date t0
epidemic_zero = epidemic_csv.loc[epidemic_csv['date'] == t0, :]
# Removing Ahvenanmaa or Aland
epidemic_zero = epidemic_zero[~epidemic_zero['erva'].str.contains('land')]

# Getting the order the ervas have inside the dataframe
ervas_order = EPIDEMIC['ervas_order']
ervas_df = list(pd.unique(epidemic_zero['erva']))
ervas_pd_order = [ervas_df.index(erva) for erva in ervas_order]

select_columns = ['susceptible',
                  'infected',
                  'exposed',
                  'recovered',
                  'vaccinated',
                  'ward',
                  'icu',
                  'infected detected',
                  'infected undetected',
                  'First dose cumulative',]
# Selecting the columns to use
epidemic_zero = epidemic_zero[select_columns]
# Converting to numpy
epidemic_npy = epidemic_zero.values
# Reshaping to 3d array
epidemic_npy = epidemic_npy.reshape(N_p, N_g, len(select_columns))

# Rearranging the order of the matrix with correct order
epidemic_npy = epidemic_npy[ervas_pd_order, :]

In [9]:
def print_latex(compartment):
    assert compartment.shape == (N_p, N_g)
    
    for age_i in range(N_g):
        for erva_i in range(N_p):
            if erva_i + 1 == N_p:
                total = np.sum(compartment[:, age_i])
                total_age = (total/np.sum(age_er[:, age_i]))*100
                print(' %.2f & %.2f & %.2f \\\\' % (round(compartment[erva_i, age_i], 2),
                                                   round(total, 2),
                                                   round(total_age, 2)), end='')
            else:
                print(' %.2f &' % (round(compartment[erva_i, age_i], 2), ), end='')
        print('')
    
    print('')
    sum_ervas = compartment.sum(axis=1)
    for erva_i in range(N_p):
        print(' %.2f &' % (round(sum_ervas[erva_i], 2), ), end='')
        if erva_i + 1 == N_p:
            print(' foo & foo \\\\')
    
    print('')
    prop_ervas = (sum_ervas/age_er.sum(axis=1))*100
    for erva_i in range(N_p):
        print(' %.2f &' % (round(prop_ervas[erva_i], 2), ), end='')
        if erva_i + 1 == N_p:
            print(' foo & \\\\')
    
    total = np.sum(compartment)
    total_prop = (total/np.sum(age_er))*100
    print("%.2f & %.2f" % (total, total_prop))

In [10]:
# v_kg
print_latex(epidemic_npy[:, :, 9])

 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\
 1802.00 & 895.00 & 647.00 & 467.00 & 397.00 & 4208.00 & 0.69 \\
 14328.00 & 6388.00 & 4803.00 & 4109.00 & 3570.00 & 33198.00 & 5.04 \\
 22284.00 & 8957.00 & 7637.00 & 6312.00 & 5640.00 & 50830.00 & 7.11 \\
 32703.00 & 12415.00 & 11717.00 & 8261.00 & 7842.00 & 72938.00 & 11.01 \\
 53120.00 & 20667.00 & 20143.00 & 15536.00 & 14676.00 & 124142.00 & 17.47 \\
 111312.00 & 46459.00 & 47326.00 & 40631.00 & 33954.00 & 279682.00 & 39.54 \\
 184417.00 & 87350.00 & 85497.00 & 79858.00 & 63630.00 & 500752.00 & 85.85 \\
 94792.00 & 50319.00 & 49239.00 & 47539.00 & 37125.00 & 279014.00 & 88.94 \\

 514758.00 & 233450.00 & 227009.00 & 202713.00 & 166834.00 & foo & foo \\

 23.42 & 26.86 & 25.15 & 25.43 & 22.65 & foo & \\
1344764.00 & 24.43


In [11]:
# S^u_kg
print_latex(epidemic_npy[:, :, 0])

 171595.42 & 70817.50 & 81907.18 & 67261.49 & 76717.83 & 468299.42 & 85.97 \\
 210496.88 & 85976.15 & 97031.70 & 81130.97 & 89006.03 & 563641.74 & 92.73 \\
 232831.91 & 91795.16 & 98109.81 & 86424.00 & 79462.98 & 588623.86 & 89.39 \\
 277544.23 & 94472.17 & 104603.96 & 84658.08 & 82551.86 & 643830.31 & 90.06 \\
 245248.97 & 89154.32 & 96534.87 & 77736.64 & 75358.04 & 584032.83 & 88.15 \\
 237874.21 & 94217.56 & 99322.28 & 91081.73 & 79726.16 & 602221.94 & 84.73 \\
 171186.79 & 79290.86 & 83915.03 & 90571.04 & 75945.60 & 500909.31 & 70.82 \\
 79687.89 & 38028.34 & 39784.10 & 39350.69 & 30833.87 & 227684.90 & 39.03 \\
 37144.58 & 20603.41 & 20835.99 & 19707.79 & 16023.24 & 114315.00 & 36.44 \\

 1663610.88 & 664355.47 & 722044.91 & 637922.42 & 605625.61 & foo & foo \\

 75.68 & 76.45 & 79.99 & 80.02 & 82.22 & foo & \\
4293559.29 & 78.01


In [12]:
# i^r_kg
print_latex(epidemic_npy[:, :, 7] + epidemic_npy[:, :, 8])

 1612.73 & 474.65 & 307.77 & 109.43 & 103.96 & 2608.54 & 0.48 \\
 688.51 & 202.64 & 131.39 & 46.72 & 44.38 & 1113.64 & 0.18 \\
 562.97 & 165.69 & 107.44 & 38.20 & 36.29 & 910.58 & 0.14 \\
 496.79 & 146.21 & 94.81 & 33.71 & 32.02 & 803.55 & 0.11 \\
 391.36 & 115.18 & 74.69 & 26.56 & 25.23 & 633.02 & 0.10 \\
 257.75 & 75.86 & 49.19 & 17.49 & 16.61 & 416.90 & 0.06 \\
 129.30 & 38.05 & 24.67 & 8.77 & 8.33 & 209.13 & 0.03 \\
 55.21 & 16.25 & 10.54 & 3.75 & 3.56 & 89.30 & 0.02 \\
 38.64 & 11.37 & 7.37 & 2.62 & 2.49 & 62.50 & 0.02 \\

 4233.25 & 1245.92 & 807.87 & 287.24 & 272.88 & foo & foo \\

 0.19 & 0.14 & 0.09 & 0.04 & 0.04 & foo & \\
6847.16 & 0.12


In [13]:
# r^r_kg
print_latex(epidemic_npy[:, :, 3])

 48404.26 & 11519.76 & 5855.94 & 4539.03 & 3486.14 & 73805.15 & 13.55 \\
 25865.12 & 6195.55 & 3247.81 & 2708.32 & 2142.57 & 40159.38 & 6.61 \\
 29244.78 & 7138.85 & 3694.80 & 3127.12 & 2512.22 & 45717.78 & 6.94 \\
 22527.03 & 5203.81 & 2763.23 & 2279.31 & 1915.45 & 34688.83 & 4.85 \\
 17445.50 & 4017.41 & 2137.80 & 1755.16 & 1474.33 & 26830.20 & 4.05 \\
 13794.60 & 3110.75 & 1682.52 & 1410.74 & 1207.43 & 21206.04 & 2.98 \\
 6751.83 & 1529.35 & 824.89 & 699.27 & 598.56 & 10403.91 & 1.47 \\
 3231.19 & 723.73 & 398.56 & 333.74 & 288.36 & 4975.57 & 0.85 \\
 2633.95 & 531.39 & 297.37 & 262.54 & 244.63 & 3969.89 & 1.27 \\

 169898.27 & 39970.61 & 20902.92 & 17115.24 & 13869.71 & foo & foo \\

 7.73 & 4.60 & 2.32 & 2.15 & 1.88 & foo & \\
261756.74 & 4.76
